In [2]:
import sklearn
import pickle
import praw
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

#### Create the reddit instance and load the model

In [3]:
reddit = praw.Reddit(client_id='#', client_secret='#',
                     user_agent='#')
model = pickle.load(open('..\\model\\model_2.sav','rb'))

#### Function to clean the text

In [4]:
replace_with_space = re.compile('[/(){}\[\]\|@,;]')
deleteSymbol = re.compile('[^0-9a-z #+_]')
stopwords = set(stopwords.words('english'))

def cleanText(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_with_space.sub(' ', text) # replace certain symbols by space in text
    text = deleteSymbol.sub('', text) # delete symbols from text
    text = ' '.join(word for word in text.split() if word not in stopwords) # remove stopwords from text
    return text

#### Function that takes the url of reddit post as input and returns the predicted flair

In [5]:
def predictFlair(url):
    submission = reddit.submission(url = url)
    
    post = {
        "title":str(submission.title),
        "body":str(submission.selftext),
        }
    comments = ""
    for comment in submission.comments:
        comments = comments + ' ' + comment.body
    post["comments"] = comments
    data = post['title'] + ' ' + post['comments'] + ' ' + post['body']
    
    return model.predict([data])

#### Utility function to process the input file and return predicted flairs
Input format: Text file with urls to reddit posts in each line; 
Output: Dictionary with url as key and predicted flair as value

In [6]:
def predictutil(filename):
    result = {}
    with open(filename,'r') as file:
        for link in file:
            result[link.strip()] = str(predictFlair(link.strip())[0])
    return result
            

In [9]:
predictFlair('https://www.reddit.com/r/india/comments/g8dx13/venomous_whisper_of_rumor_mongers_manifest/')

array(['Politics'], dtype=object)